In [56]:
import pandas as pd

price_df = pd.read_parquet("../../data/Bloomberg Data Retrive.parquet")
# Filter the DataFrame by 'PLLFLNJK PLDP Index'
price_df = price_df[price_df['Name'] == 'PLLFLNJK PLDP Index']
# Drop the columns
price_df = price_df.drop(columns=['VOLUME', 'PX_ASK', 'PX_BID', 'PX_SETTLE', 'OPEN_INT', 'Name'])

In [57]:
price_df = price_df.iloc[0:3802]
# price_df
# price_df.to_csv("../../data/Fine-Tuning/PLLFLNJK_PLDP_Index_Price.csv", index=False)

In [58]:
price_df['return'] = price_df['last_price'].pct_change()
price_df['volatility'] = price_df['return'].rolling(window=252).std()
price_df['next_day_price'] = price_df['last_price'].shift(-1)
price_df['T+3 return'] = price_df['next_day_price'].pct_change(3)
price_df['T+5 return'] = price_df['next_day_price'].pct_change(5)
price_df['T+10 return'] = price_df['next_day_price'].pct_change(10)
price_df['volatility_weekly'] = price_df['return'].rolling(window=5).std()

# price_df.to_csv("../../data/Fine-Tuning/PLLFLNJK_PLDP_FULL.csv", index=False)


In [59]:
news_path = '../../data/WSJ/WSJ_data_modified.csv'
news = pd.read_csv(news_path)
wsj_data_modified_reversed = news.iloc[::-1].reset_index(drop=True)

# data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.strftime('%Y-%m-%d')

In [60]:
from datetime import timedelta

# Convert 'Timestamp' column to datetime
wsj_data_modified_reversed['Timestamp'] = pd.to_datetime(wsj_data_modified_reversed['Timestamp'])

# Function to adjust date to the Friday of the same week if it's a weekend
def adjust_to_friday(date):
    if date.weekday() == 5:  # Saturday
        return date - timedelta(days=1)
    elif date.weekday() == 6:  # Sunday
        return date - timedelta(days=2)
    else:
        return date

# Apply the function to the 'Timestamp' column
wsj_data_modified_reversed['Timestamp'] = wsj_data_modified_reversed['Timestamp'].apply(adjust_to_friday)
wsj_data_modified_reversed


,Timestamp,Title,Theme,Summary
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...
4,2020-05-12,The Daily Shot: Tepid Signs of Stabilization i...,Dailyshot Blog,NaN
...,...,...,...,...
554,2024-03-14,Both Parties Go After Robert Hur's Report on C...,PODCAST,Special counsel Robert Hur received criticism ...
555,2024-03-29,The Climate Scientist Fossil-Fuel Companies Ca...,Gas Markets,Robert Howarth’s research on natural gas expor...
556,2024-03-29,Opinion: Green Shareholders Try to Sabotage Am...,Commentary,A wave of destructive proposals began after a ...
557,2024-04-02,Natural Gas Pioneer Charif Souki Ordered to Pa...,Pro Bankruptcy,The ruling stems from disputes with lenders wh...


In [61]:
# Convert the 'Date' columns in both dataframes to datetime for accurate merging
price_df['Date'] = pd.to_datetime(price_df['Date'])
wsj_data_modified_reversed['Timestamp'] = pd.to_datetime(wsj_data_modified_reversed['Timestamp'])

# Perform a left join of wsj_data_modified_reversed with pllflnjk_pldp_index_data based on the 'Timestamp'/'Date' columns
merged_data = wsj_data_modified_reversed.merge(price_df, left_on='Timestamp', right_on='Date', how='left')

# Since we're interested in adding columns from PLLFLNJK_PLDP_Index to WSJ_data_modified, we'll drop the 'Date' column from the result to avoid duplication
merged_data.drop('Date', axis=1, inplace=True)
merged_data


,Timestamp,Title,Theme,Summary,last_price,return,volatility,next_day_price
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...,2.186,-0.084590,0.032920,2.175
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...,2.175,-0.005032,0.032737,2.040
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...,2.040,-0.062069,0.032822,2.000
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...,1.900,-0.012987,0.032841,1.825
4,2020-05-12,The Daily Shot: Tepid Signs of Stabilization i...,Dailyshot Blog,NaN,2.438,0.048602,0.034201,2.450
...,...,...,...,...,...,...,...,...
554,2024-03-14,Both Parties Go After Robert Hur's Report on C...,PODCAST,Special counsel Robert Hur received criticism ...,8.792,0.008951,0.044283,9.142
555,2024-03-29,The Climate Scientist Fossil-Fuel Companies Ca...,Gas Markets,Robert Howarth’s research on natural gas expor...,NaN,NaN,NaN,NaN
556,2024-03-29,Opinion: Green Shareholders Try to Sabotage Am...,Commentary,A wave of destructive proposals began after a ...,NaN,NaN,NaN,NaN
557,2024-04-02,Natural Gas Pioneer Charif Souki Ordered to Pa...,Pro Bankruptcy,The ruling stems from disputes with lenders wh...,9.383,-0.011692,0.044284,NaN


In [67]:
# merged_data.to_csv("../../data/Fine-Tuning/merged_raw.csv", index=False)
cleaned_df = merged_data.dropna(subset=['Summary', 'last_price'])
cleaned_df = cleaned_df[cleaned_df['return'] != 0]
cleaned_df
# cleaned_df.to_csv("../../data/Fine-Tuning/merged_cleaned.csv", index=False)

In [71]:
# Grouping by Timestamp and merging the Title, Theme, and Summary columns as specified
# If there's only one record for a day, it will be left as is, otherwise they will be merged with numbering

# Define a custom function to merge columns with numbering if there are multiple records, else leave as is
def merge_with_numbering(series):
    if len(series) == 1:
        return series.iloc[0]
    else:
        return ''.join(f"({i+1}) {x} " for i, x in enumerate(series))

# Group by 'Timestamp' and apply the custom function to 'Title', 'Theme', and 'Summary'
grouped = cleaned_df.groupby('Timestamp').agg({
    'Title': merge_with_numbering,
    'Theme': merge_with_numbering,
    'Summary': merge_with_numbering,
    'last_price': 'first',  # Keeping the rest of the columns the same (assuming they're identical for grouped records)
    'return': 'first',
    'volatility': 'first',
    'next_day_price': 'first'
}).reset_index()

# Display the first few rows to verify the transformation
grouped

,Timestamp,Title,Theme,Summary,last_price,return,volatility,next_day_price
0,2020-04-17,Fall of Natural Gas Prices Speeds Energy Shift...,Gas Markets,When natural gas became as cheap as coal in th...,2.186,-0.084590,0.032920,2.175
1,2020-04-20,Coronavirus Contract Disputes Start Hitting th...,U.S.,Companies are suspending or terminating busine...,2.175,-0.005032,0.032737,2.040
2,2020-04-21,Shell Delays $1 Billion North Sea Development,Business,Royal Dutch Shell has postponed its Jackdaw na...,2.040,-0.062069,0.032822,2.000
3,2020-04-27,Ship Orders Crash as Coronavirus Takes a Toll ...,Logistics Report,The economic fallout from coronavirus restrict...,1.900,-0.012987,0.032841,1.825
4,2020-05-21,A Natural Gas Exec Turned His Colorado Ranch I...,Homes,Egypt-born entrepreneur Charif Souki invested ...,2.125,-0.022089,0.034200,1.887
...,...,...,...,...,...,...,...,...
319,2024-02-23,(1) Corrections & Amplifications (2) Natural G...,(1) Corrections (2) Gas Markets,(1) Corrections & Amplifications for the editi...,7.983,-0.030012,0.045362,7.981
320,2024-03-11,Elliott Bets $500 Million on Gas Producer Back...,Pro PE Fund News,Paul Singer’s Elliott Investment is plowing mo...,8.802,-0.038873,0.045546,8.553
321,2024-03-13,Biggest Gains in Financial Strength in the Man...,Journal Reports: Leadership,"Royal Caribbean heads the list, followed by Ch...",8.714,0.018824,0.044321,8.792
322,2024-03-14,(1) Shell to Slow Pace of Carbon-Emission Cuts...,(1) Sustainable Business (2) PODCAST,(1) Anglo-Dutch energy heavyweight said it now...,8.792,0.008951,0.044283,9.142


In [62]:
# price_df['Date'] = pd.to_datetime(price_df['Date'])
# unique_elements = price_df['Name'].unique()
# unique_elements